In [1]:
import warnings
from graspy.utils import import_graph, to_laplacian, is_fully_connected

ImportError: cannot import name 'to_laplacian' from 'graspy.utils' (/Users/danielborders/opt/anaconda3/lib/python3.7/site-packages/graspy/utils/__init__.py)

In [33]:
from abc import abstractmethod

import numpy as np
from sklearn.base import BaseEstimator

from graspy.utils import augment_diagonal, import_graph, is_almost_symmetric
from graspy.embed.svd import selectSVD

from graspy.simulations import sbm

from graspy.embed.utils import import_graph, to_laplacian, is_fully_connected

ModuleNotFoundError: No module named 'graspy.embed.utils'

In [7]:
class BaseSpectralEmbed(BaseEstimator):
    def __init__(
        self,
        n_components=None,
        n_elbows=2,
        algorithm="randomized",
        n_iter=5,
        check_lcc=True,
        concat=False,
    ):
        self.n_components = n_components
        self.n_elbows = n_elbows
        self.algorithm = algorithm
        self.n_iter = n_iter
        self.check_lcc = check_lcc
        if not isinstance(concat, bool):
            msg = "Parameter `concat` is expected to be type bool"
            raise TypeError(msg)
        self.concat = concat

    def _reduce_dim(self, A):
        U, D, V = selectSVD(
            A,
            n_components=self.n_components,
            n_elbows=self.n_elbows,
            algorithm=self.algorithm,
            n_iter=self.n_iter,
        )

        self.n_components_ = D.size
        self.singular_values_ = D
        self.latent_left_ = U @ np.diag(np.sqrt(D))
        if not is_almost_symmetric(A):
            self.latent_right_ = V.T @ np.diag(np.sqrt(D))
        else:
            self.latent_right_ = None

    @property
    def _pairwise(self):
        """This is for sklearn compliance."""
        return True

    @abstractmethod
    def fit(self, graph, y=None):
        # call self._reduce_dim(A) from your respective embedding technique.
        # import graph(s) to an adjacency matrix using import_graph function
        # here

        return self

    def _fit_transform(self, graph):
        "Fits the model and returns the estimated latent positions."
        self.fit(graph)

        if self.latent_right_ is None:
            return self.latent_left_
        else:
            if self.concat:
                return np.concatenate((self.latent_left_, self.latent_right_), axis=1)
            else:
                return self.latent_left_, self.latent_right_

    def fit_transform(self, graph, y=None):
        return self._fit_transform(graph)


class BaseEmbedMulti(BaseSpectralEmbed):
    def __init__(
        self,
        n_components=None,
        n_elbows=2,
        algorithm="randomized",
        n_iter=5,
        check_lcc=True,
        diag_aug=True,
        concat=False,
    ):
        super().__init__(
            n_components=n_components,
            n_elbows=n_elbows,
            algorithm=algorithm,
            n_iter=n_iter,
            check_lcc=check_lcc,
            concat=concat,
        )

        if not isinstance(diag_aug, bool):
            raise TypeError("`diag_aug` must be of type bool")
        self.diag_aug = diag_aug

    def _check_input_graphs(self, graphs):
        # Convert input to np.arrays
        # This check is needed because np.stack will always duplicate array in memory.
        if isinstance(graphs, (list, tuple)):
            if len(graphs) <= 1:
                msg = "Input {} must have at least 2 graphs, not {}.".format(
                    type(graphs), len(graphs)
                )
                raise ValueError(msg)
            out = [import_graph(g, copy=False) for g in graphs]
        elif isinstance(graphs, np.ndarray):
            if graphs.ndim != 3:
                msg = "Input tensor must be 3-dimensional, not {}-dimensional.".format(
                    graphs.ndim
                )
                raise ValueError(msg)
            elif graphs.shape[0] <= 1:
                msg = "Input tensor must have at least 2 elements, not {}.".format(
                    graphs.shape[0]
                )
                raise ValueError(msg)
            out = import_graph(graphs, copy=False)
        else:
            msg = "Input must be a list or ndarray, not {}.".format(type(graphs))
            raise TypeError(msg)

        # Save attributes
        self.n_graphs_ = len(out)
        self.n_vertices_ = out[0].shape[0]

        return out

    def _diag_aug(self, graphs):
        if isinstance(graphs, list):
            out = [augment_diagonal(g) for g in graphs]
        elif isinstance(graphs, np.ndarray):
            # Copying is necessary to not overwrite input array
            out = np.array([augment_diagonal(graphs[i]) for i in range(self.n_graphs_)])

        return out

In [8]:
class LaplacianSpectralEmbed(BaseSpectralEmbed):
    def __init__(
        self,
        form="DAD",
        n_components=None,
        n_elbows=2,
        algorithm="randomized",
        n_iter=5,
        check_lcc=True,
        regularizer=None,
        concat=False,
    ):
        super().__init__(
            n_components=n_components,
            n_elbows=n_elbows,
            algorithm=algorithm,
            n_iter=n_iter,
            check_lcc=check_lcc,
            concat=concat,
        )
        self.form = form
        self.regularizer = regularizer

    def fit(self, graph, y=None):
        A = import_graph(graph)

        if self.check_lcc:
            if not is_fully_connected(A):
                msg = (
                    "Input graph is not fully connected. Results may not"
                    + "be optimal. You can compute the largest connected component by"
                    + "using ``graspologic.utils.largest_connected_component``."
                )
                warnings.warn(msg, UserWarning)

        self.n_features_in_ = A.shape[0]
        L_norm = to_laplacian(A, form=self.form, regularizer=self.regularizer)
        self._reduce_dim(L_norm)
        return self

In [16]:
from graspy.embed import LaplacianSpectralEmbed
import numpy as np
from graspy.simulations import sbm
import numpy as np

In [17]:
lse = LaplacianSpectralEmbed(n_components=2)

In [18]:
n_verts = 250
labels_sbm = n_verts * [0] + n_verts * [1]
P = np.array([[0.8, 0.2],
              [0.2, 0.8]])

latents = np.repeat(P, n_verts, axis=0)

undirected_sbm = sbm(2 * [n_verts], P)

In [21]:
in_sample = np.arange(n_verts*2 - 1)
A = undirected_sbm[np.ix_(in_sample, in_sample)]

In [25]:
oos_idx = np.array([499])
ossA = undirected_sbm[np.ix_(oos_idx, in_sample)]

In [29]:
lse_fit = lse.fit(A)

In [30]:
U = ossA @ lse.latent_left_ @ np.diag(1/np.sqrt(lse.singular_values_)) @ np.diag(1/np.sqrt(lse.singular_values_)) 

In [31]:
U

array([[11.01342157, -8.47381614]])

In [32]:
lse.latent_left_

array([[ 0.04449384,  0.03401002],
       [ 0.04422059,  0.03701839],
       [ 0.04440295,  0.03643569],
       [ 0.04548172,  0.03249927],
       [ 0.04376138,  0.03586168],
       [ 0.04449384,  0.03263479],
       [ 0.04494557,  0.03454603],
       [ 0.04512499,  0.03399696],
       [ 0.04485559,  0.03331115],
       [ 0.04412913,  0.03376916],
       [ 0.04431186,  0.03759155],
       [ 0.04449384,  0.03686463],
       [ 0.04339052,  0.03609777],
       [ 0.04376138,  0.03295883],
       [ 0.04521444,  0.03679944],
       [ 0.04476542,  0.03366161],
       [ 0.04601162,  0.03224831],
       [ 0.04385361,  0.03285544],
       [ 0.04557046,  0.03502653],
       [ 0.04412913,  0.03486789],
       [ 0.04449384,  0.03452569],
       [ 0.04503537,  0.03640925],
       [ 0.04467508,  0.03413741],
       [ 0.04394564,  0.03784681],
       [ 0.04422059,  0.03166496],
       [ 0.04467508,  0.03371386],
       [ 0.04601162,  0.03419707],
       [ 0.04557046,  0.03679084],
       [ 0.04574744,